In [2]:
from library.common import Database
import pandas as pd

db = Database()
db_table = 'owid_co2_greenhouse_gas_emissions'
owid_df = pd.read_sql(db_table, db.config)

owid_df
full_features = list(owid_df.columns)
#drop continuents
owid_df = owid_df.dropna(subset = ['iso_code']).copy()

print("Data loaded")

Data loaded


In [3]:
# selected = \
#     ['iso_code',
#      'year',
#      'co2',
#      'consumption_co2',
#      'co2_growth_abs',
#      'trade_co2',
#      'co2_per_gdp',
#      'co2_per_unit_energy',
#      'coal_co2',
#      'cement_co2',
#      'flaring_co2',
#      'gas_co2',
#      'oil_co2',
#      'other_industry_co2',
#      'population',
#      'gdp',
#      'primary_energy_consumption']

selected = ["iso_code", "year","trade_co2"]


In [4]:
dataset = owid_df[selected].copy()

dataset = dataset.fillna(0)
global_df = dataset.groupby('year').sum().reset_index(drop = False)
timeseries = global_df[:-2].copy()

In [5]:
import numpy as np
import pmdarima as pm

def get_optimal_param(timeseries):
    stepwise_fit = pm.auto_arima(timeseries,
                                 max_p=20,
                                 max_d=20,
                                 max_q=20,
                                 seasonal=False,
                                 error_action='ignore',
                                 suppress_warnings=True,
                                 trace =  True,
                                 stepwise=True)  # set to stepwise
    return stepwise_fit.get_params().get('order')

# this is a dataset from R
# wineind = load_wineind().astype(np.float64)

result = dict()
for feature in selected[2:]:
    print(feature)
    series = dataset[feature].to_numpy()
    result[feature] = get_optimal_param(series)

print(result)

trade_co2
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=192355.601, Time=3.23 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=227493.055, Time=0.27 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=193296.585, Time=0.42 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=209637.350, Time=0.82 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=192360.700, Time=1.50 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=193178.856, Time=2.39 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=192374.076, Time=2.53 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=192296.718, Time=6.99 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=192357.318, Time=2.32 sec
 ARIMA(3,0,3)(0,0,0)[0]             : AIC=192295.045, Time=8.35 sec
 ARIMA(4,0,3)(0,0,0)[0]             : AIC=192272.104, Time=12.88 sec
 ARIMA(4,0,2)(0,0,0)[0]             : AIC=192277.597, Time=8.48 sec


KeyboardInterrupt: 

In [ ]:
import json

with open('arima_parametersX.json', 'w') as f:
    json.dump(result, f)